In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Graphics
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling
import lightgbm as lgb
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import f1_score, recall_score, accuracy_score, precision_score
from sklearn.metrics import classification_report, log_loss
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix

# Divers
from time import time, strftime, gmtime
import gc
import pickle

# Module des fonctions du notebook
import fonctions08 as f

# Autoreload pour prise en compte des changments dans le module fonctions
%load_ext autoreload
%autoreload 1
%aimport fonctions08

In [2]:
# Heure démarrage
t0=time()

# Constantes
RANDOM_STATE = 1
NROWS = None
NCOLS = None
N_FOLDS = 5

# Paramètres pour les validations croisées
random_seed = 1 # Seed pour générateur de nombre aléatoire
folds = 5 # Nombre de folds pour validation croisée

In [3]:
features = f.import_csv("data/features/train_features_selected.csv", nrows=NROWS)
target = f.import_csv("data/features/train_target.csv", nrows=NROWS)

features = features.set_index("SK_ID_CURR")
target = target.set_index("SK_ID_CURR")

print(features.shape)
print(target.shape)

Memory usage of dataframe is 505.82 MB
Memory usage after optimization is: 164.84 MB
Decreased by 67.4%
Memory usage of dataframe is 3.28 MB
Memory usage after optimization is: 1.03 MB
Decreased by 68.7%
(215257, 307)
(215257, 1)


# Echantillon avec classes équilibrées

In [4]:
target[target['TARGET'] == 1].shape

(17377, 1)

In [5]:
target_0 = target[target['TARGET'] == 0]
target_1 = target[target['TARGET'] == 1]

target_0_sample = target_0.sample(len(target_1), random_state=RANDOM_STATE)

target_sample_balanced = pd.concat([target_0_sample, target_1], axis=0)
features_sample_balanced = features.loc[target_sample_balanced.index, :].iloc[:, :NCOLS]

In [6]:
features_sample_balanced.shape

(34754, 307)

In [7]:
# Creation d'un train set de 10000 observations et test set de 6000 obs
X_train, X_test, y_train, y_test =\
    train_test_split(features_sample_balanced, target_sample_balanced,
                     train_size=15000, test_size=6000,
                     stratify=target_sample_balanced, random_state=RANDOM_STATE)

# TEST CV + LGBM + F1 SCORE

In [8]:
import optuna

def objective(trial):
    params = {
        'verbosity':-1,
        'objective': 'binary',
        'metric': 'auc',        
        'boosting_type': 'gbdt',
        'n_estimators': trial.suggest_int('n_estimators', 20, 200),
        'num_leaves ': trial.suggest_int('num_leaves', 8, 128),
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0.0, 1.0),
    }
    
    k_fold = KFold(n_splits=N_FOLDS, shuffle=False, random_state=RANDOM_STATE)
    
    scores=[]

    for train_index, valid_index in k_fold.split(X_train):
    
        # Training data for the fold
        fold_X_train, fold_y_train = X_train.iloc[train_index, :], y_train.iloc[train_index, :]
        
        # Validation data for the fold
        fold_X_valid, fold_y_valid = X_train.iloc[valid_index, :], y_train.iloc[valid_index, :]
        
        model = lgb.LGBMModel(random_state=1)
        model.set_params(**params)
        model.fit(fold_X_train, np.ravel(fold_y_train))
        preds = model.predict(fold_X_valid)
        scores.append(log_loss(fold_y_valid, preds))
   
    return np.mean(scores)

In [9]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1000, timeout=None) # timeout (seconds)

[I 2020-01-19 00:49:05,746] Finished trial#0 resulted in value: 0.5943931030002678. Current best value is 0.5943931030002678 with parameters: {'n_estimators': 43, 'num_leaves': 57, 'max_depth': 8, 'learning_rate': 0.048049384074491774, 'min_child_samples': 35, 'colsample_bytree': 0.9703254052241811, 'reg_alpha': 0.3301781126384886}.
[I 2020-01-19 00:49:13,981] Finished trial#1 resulted in value: 0.594740305103209. Current best value is 0.5943931030002678 with parameters: {'n_estimators': 43, 'num_leaves': 57, 'max_depth': 8, 'learning_rate': 0.048049384074491774, 'min_child_samples': 35, 'colsample_bytree': 0.9703254052241811, 'reg_alpha': 0.3301781126384886}.
[I 2020-01-19 00:49:18,770] Finished trial#2 resulted in value: 0.5928976260574816. Current best value is 0.5928976260574816 with parameters: {'n_estimators': 94, 'num_leaves': 36, 'max_depth': 4, 'learning_rate': 0.032230628124461445, 'min_child_samples': 69, 'colsample_bytree': 0.5073009156385371, 'reg_alpha': 0.148919162422700

[I 2020-01-19 00:55:20,654] Finished trial#24 resulted in value: 0.5740539299029938. Current best value is 0.5731713637532534 with parameters: {'n_estimators': 190, 'num_leaves': 27, 'max_depth': 9, 'learning_rate': 0.039042929794182314, 'min_child_samples': 13, 'colsample_bytree': 0.8138439458916014, 'reg_alpha': 0.9742093372205706}.
[I 2020-01-19 00:55:39,556] Finished trial#25 resulted in value: 0.575002589749348. Current best value is 0.5731713637532534 with parameters: {'n_estimators': 190, 'num_leaves': 27, 'max_depth': 9, 'learning_rate': 0.039042929794182314, 'min_child_samples': 13, 'colsample_bytree': 0.8138439458916014, 'reg_alpha': 0.9742093372205706}.
[I 2020-01-19 00:55:58,906] Finished trial#26 resulted in value: 0.5738968320045125. Current best value is 0.5731713637532534 with parameters: {'n_estimators': 190, 'num_leaves': 27, 'max_depth': 9, 'learning_rate': 0.039042929794182314, 'min_child_samples': 13, 'colsample_bytree': 0.8138439458916014, 'reg_alpha': 0.974209337

[I 2020-01-19 01:02:50,597] Finished trial#48 resulted in value: 0.6394165713324576. Current best value is 0.5731713637532534 with parameters: {'n_estimators': 190, 'num_leaves': 27, 'max_depth': 9, 'learning_rate': 0.039042929794182314, 'min_child_samples': 13, 'colsample_bytree': 0.8138439458916014, 'reg_alpha': 0.9742093372205706}.
[I 2020-01-19 01:03:09,340] Finished trial#49 resulted in value: 0.5742455131424278. Current best value is 0.5731713637532534 with parameters: {'n_estimators': 190, 'num_leaves': 27, 'max_depth': 9, 'learning_rate': 0.039042929794182314, 'min_child_samples': 13, 'colsample_bytree': 0.8138439458916014, 'reg_alpha': 0.9742093372205706}.
[I 2020-01-19 01:03:26,252] Finished trial#50 resulted in value: 0.5768313484609162. Current best value is 0.5731713637532534 with parameters: {'n_estimators': 190, 'num_leaves': 27, 'max_depth': 9, 'learning_rate': 0.039042929794182314, 'min_child_samples': 13, 'colsample_bytree': 0.8138439458916014, 'reg_alpha': 0.97420933

[I 2020-01-19 01:11:14,562] Finished trial#72 resulted in value: 0.5736380370490217. Current best value is 0.5731443329066443 with parameters: {'n_estimators': 192, 'num_leaves': 85, 'max_depth': 9, 'learning_rate': 0.040579350558634056, 'min_child_samples': 93, 'colsample_bytree': 0.924516216129443, 'reg_alpha': 0.7871594463511955}.
[I 2020-01-19 01:11:38,727] Finished trial#73 resulted in value: 0.573996921243592. Current best value is 0.5731443329066443 with parameters: {'n_estimators': 192, 'num_leaves': 85, 'max_depth': 9, 'learning_rate': 0.040579350558634056, 'min_child_samples': 93, 'colsample_bytree': 0.924516216129443, 'reg_alpha': 0.7871594463511955}.
[I 2020-01-19 01:12:00,640] Finished trial#74 resulted in value: 0.5735093973707714. Current best value is 0.5731443329066443 with parameters: {'n_estimators': 192, 'num_leaves': 85, 'max_depth': 9, 'learning_rate': 0.040579350558634056, 'min_child_samples': 93, 'colsample_bytree': 0.924516216129443, 'reg_alpha': 0.787159446351

[I 2020-01-19 01:19:22,967] Finished trial#96 resulted in value: 0.575195997760671. Current best value is 0.5731443329066443 with parameters: {'n_estimators': 192, 'num_leaves': 85, 'max_depth': 9, 'learning_rate': 0.040579350558634056, 'min_child_samples': 93, 'colsample_bytree': 0.924516216129443, 'reg_alpha': 0.7871594463511955}.
[I 2020-01-19 01:19:42,919] Finished trial#97 resulted in value: 0.5748067521405633. Current best value is 0.5731443329066443 with parameters: {'n_estimators': 192, 'num_leaves': 85, 'max_depth': 9, 'learning_rate': 0.040579350558634056, 'min_child_samples': 93, 'colsample_bytree': 0.924516216129443, 'reg_alpha': 0.7871594463511955}.
[I 2020-01-19 01:20:05,480] Finished trial#98 resulted in value: 0.5750173107180181. Current best value is 0.5731443329066443 with parameters: {'n_estimators': 192, 'num_leaves': 85, 'max_depth': 9, 'learning_rate': 0.040579350558634056, 'min_child_samples': 93, 'colsample_bytree': 0.924516216129443, 'reg_alpha': 0.787159446351

...

[I 2020-01-19 04:34:37,069] Finished trial#888 resulted in value: 0.5725349967013129. Current best value is 0.5712899767520222 with parameters: {'n_estimators': 200, 'num_leaves': 106, 'max_depth': 9, 'learning_rate': 0.04245437262781256, 'min_child_samples': 89, 'colsample_bytree': 0.5406006780633795, 'reg_alpha': 0.3473587828143288}.
[I 2020-01-19 04:34:53,502] Finished trial#889 resulted in value: 0.5729914304938566. Current best value is 0.5712899767520222 with parameters: {'n_estimators': 200, 'num_leaves': 106, 'max_depth': 9, 'learning_rate': 0.04245437262781256, 'min_child_samples': 89, 'colsample_bytree': 0.5406006780633795, 'reg_alpha': 0.3473587828143288}.
[I 2020-01-19 04:35:09,014] Finished trial#890 resulted in value: 0.5735223136328058. Current best value is 0.5712899767520222 with parameters: {'n_estimators': 200, 'num_leaves': 106, 'max_depth': 9, 'learning_rate': 0.04245437262781256, 'min_child_samples': 89, 'colsample_bytree': 0.5406006780633795, 'reg_alpha': 0.34735

[I 2020-01-19 04:40:49,555] Finished trial#912 resulted in value: 0.572836167080429. Current best value is 0.5712899767520222 with parameters: {'n_estimators': 200, 'num_leaves': 106, 'max_depth': 9, 'learning_rate': 0.04245437262781256, 'min_child_samples': 89, 'colsample_bytree': 0.5406006780633795, 'reg_alpha': 0.3473587828143288}.
[I 2020-01-19 04:41:06,426] Finished trial#913 resulted in value: 0.5716132397167296. Current best value is 0.5712899767520222 with parameters: {'n_estimators': 200, 'num_leaves': 106, 'max_depth': 9, 'learning_rate': 0.04245437262781256, 'min_child_samples': 89, 'colsample_bytree': 0.5406006780633795, 'reg_alpha': 0.3473587828143288}.
[I 2020-01-19 04:41:22,644] Finished trial#914 resulted in value: 0.5729902338254228. Current best value is 0.5712899767520222 with parameters: {'n_estimators': 200, 'num_leaves': 106, 'max_depth': 9, 'learning_rate': 0.04245437262781256, 'min_child_samples': 89, 'colsample_bytree': 0.5406006780633795, 'reg_alpha': 0.347358

[I 2020-01-19 04:47:13,280] Finished trial#936 resulted in value: 0.5744052773606316. Current best value is 0.5712899767520222 with parameters: {'n_estimators': 200, 'num_leaves': 106, 'max_depth': 9, 'learning_rate': 0.04245437262781256, 'min_child_samples': 89, 'colsample_bytree': 0.5406006780633795, 'reg_alpha': 0.3473587828143288}.
[I 2020-01-19 04:47:30,036] Finished trial#937 resulted in value: 0.5724098050259204. Current best value is 0.5712899767520222 with parameters: {'n_estimators': 200, 'num_leaves': 106, 'max_depth': 9, 'learning_rate': 0.04245437262781256, 'min_child_samples': 89, 'colsample_bytree': 0.5406006780633795, 'reg_alpha': 0.3473587828143288}.
[I 2020-01-19 04:47:46,189] Finished trial#938 resulted in value: 0.5730610763173243. Current best value is 0.5712899767520222 with parameters: {'n_estimators': 200, 'num_leaves': 106, 'max_depth': 9, 'learning_rate': 0.04245437262781256, 'min_child_samples': 89, 'colsample_bytree': 0.5406006780633795, 'reg_alpha': 0.34735

[I 2020-01-19 04:53:34,955] Finished trial#960 resulted in value: 0.5733207532918043. Current best value is 0.5712899767520222 with parameters: {'n_estimators': 200, 'num_leaves': 106, 'max_depth': 9, 'learning_rate': 0.04245437262781256, 'min_child_samples': 89, 'colsample_bytree': 0.5406006780633795, 'reg_alpha': 0.3473587828143288}.
[I 2020-01-19 04:53:50,607] Finished trial#961 resulted in value: 0.5741552896742311. Current best value is 0.5712899767520222 with parameters: {'n_estimators': 200, 'num_leaves': 106, 'max_depth': 9, 'learning_rate': 0.04245437262781256, 'min_child_samples': 89, 'colsample_bytree': 0.5406006780633795, 'reg_alpha': 0.3473587828143288}.
[I 2020-01-19 04:54:06,112] Finished trial#962 resulted in value: 0.574294256551654. Current best value is 0.5712899767520222 with parameters: {'n_estimators': 200, 'num_leaves': 106, 'max_depth': 9, 'learning_rate': 0.04245437262781256, 'min_child_samples': 89, 'colsample_bytree': 0.5406006780633795, 'reg_alpha': 0.347358

[I 2020-01-19 04:59:47,693] Finished trial#984 resulted in value: 0.5735632790492229. Current best value is 0.5712899767520222 with parameters: {'n_estimators': 200, 'num_leaves': 106, 'max_depth': 9, 'learning_rate': 0.04245437262781256, 'min_child_samples': 89, 'colsample_bytree': 0.5406006780633795, 'reg_alpha': 0.3473587828143288}.
[I 2020-01-19 05:00:04,438] Finished trial#985 resulted in value: 0.5734846808928145. Current best value is 0.5712899767520222 with parameters: {'n_estimators': 200, 'num_leaves': 106, 'max_depth': 9, 'learning_rate': 0.04245437262781256, 'min_child_samples': 89, 'colsample_bytree': 0.5406006780633795, 'reg_alpha': 0.3473587828143288}.
[I 2020-01-19 05:00:21,153] Finished trial#986 resulted in value: 0.5732499057647018. Current best value is 0.5712899767520222 with parameters: {'n_estimators': 200, 'num_leaves': 106, 'max_depth': 9, 'learning_rate': 0.04245437262781256, 'min_child_samples': 89, 'colsample_bytree': 0.5406006780633795, 'reg_alpha': 0.34735

In [10]:
print('Number of finished trials: {}'.format(len(study.trials)))

print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))

print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

Number of finished trials: 1000
Best trial:
  Value: 0.5712757618673244
  Params: 
    n_estimators: 200
    num_leaves: 68
    max_depth: 9
    learning_rate: 0.04103172930436684
    min_child_samples: 37
    colsample_bytree: 0.5286478067094085
    reg_alpha: 0.034909609792110274


In [11]:
# Modele final
model = lgb.LGBMModel(objective='binary', metric='auc',
                      boosting_type= 'gbdt')
model.set_params(**study.best_params)
model

LGBMModel(boosting_type='gbdt', class_weight=None,
     colsample_bytree=0.5286478067094085, importance_type='split',
     learning_rate=0.04103172930436684, max_depth=9, metric='auc',
     min_child_samples=37, min_child_weight=0.001, min_split_gain=0.0,
     n_estimators=200, n_jobs=-1, num_leaves=68, objective='binary',
     random_state=None, reg_alpha=0.034909609792110274, reg_lambda=0.0,
     silent=True, subsample=1.0, subsample_for_bin=200000,
     subsample_freq=0)

In [12]:
# On sauvegarde le modèle et la liste des features
model_params = {'model': model.get_params(), 'study': study}
filename = 'data/model/params_log_loss.sav'
pickle.dump(model_params, open(filename, 'wb'))

In [13]:
# Aperçu du dataframe des combinaisons de valeurs testées
df_res = pickle.load(open(filename, 'rb'))['study'].trials_dataframe()
df_res.head()

number                state     value             datetime_start  \
                                                                     
0      0  TrialState.COMPLETE  0.594393 2020-01-19 00:48:57.558640   
1      1  TrialState.COMPLETE  0.594740 2020-01-19 00:49:05.746771   
2      2  TrialState.COMPLETE  0.592898 2020-01-19 00:49:13.981742   
3      3  TrialState.COMPLETE  0.579558 2020-01-19 00:49:18.770958   
4      4  TrialState.COMPLETE  0.588416 2020-01-19 00:49:26.017845   

           datetime_complete           params                          \
                             colsample_bytree learning_rate max_depth   
0 2020-01-19 00:49:05.746771         0.970325      0.048049         8   
1 2020-01-19 00:49:13.981742         0.771245      0.031128        10   
2 2020-01-19 00:49:18.770958         0.507301      0.032231         4   
3 2020-01-19 00:49:26.017845         0.610260      0.074084         8   
4 2020-01-19 00:49:31.149128         0.961792      0.062350         4   

                                                      system_attrs  
  min_child_samples n_estimators num_leaves reg_alpha      _number  
0                35           43         57  0.330178            0  
1                97           64         73  0.671019            1  
2                69           94         36  0.148919            2  
3                38           63         78  0.325553            3  
4                18           63         47  0.814750            4

In [14]:
t1 = time()
print("computing time : {:8.6f} sec".format(t1-t0))
print("computing time : " + strftime('%H:%M:%S', gmtime(t1-t0)))

computing time : 15349.936820 sec
computing time : 04:15:49
